# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url, AgentState

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            2.6.1
OpenAI Agents SDK  0.4.2
Pydantic           2.12.3


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# turn off open telemetry (langfuse)
os.environ["OTEL_SDK_DISABLED"] = "true"

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger
debuglevel = logging.DEBUG

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig()
    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(debuglevel)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(debuglevel)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(debuglevel)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.debug("Test debug message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

09:40:40 | NewsletterAgent.newsletter_agent | DEBUG | Test debug message
09:40:40 | NewsletterAgent.newsletter_agent | INFO | Test info message
09:40:40 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
09:40:40 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

In [5]:
# look up recent sessions
conn = sqlite3.connect("newsletter_agent.db")
AgentState.list_sessions(conn, n_records=10)


['test_newsletter_20260105071737715358',
 'test_newsletter_20260104105906291196',
 'test_newsletter_20260103183740760295',
 'test_newsletter_20260103102050913766',
 'test_newsletter_20260102184758171914',
 'test_newsletter_20260102091524914959',
 'test_newsletter_20260101094717013802',
 'test_newsletter_20251231182839100574',
 'test_newsletter_20251231093009469841',
 'test_newsletter_20251230091653525056']

In [6]:
z = AgentState.get_latest_by_session(conn, "test_newsletter_20260104105906291196")
z.step_name

'cluster_topics'

In [7]:
for y in z.get_all_by_session(conn, "test_newsletter_20260104105906291196"):
    print(y.session_id, y.step_name, y.updated_at)

test_newsletter_20260104105906291196 initialize 2026-01-04 10:59:06.312236
test_newsletter_20260104105906291196 gather_urls 2026-01-04 11:00:03.281455
test_newsletter_20260104105906291196 check_workflow_status 2026-01-04 11:00:05.146237
test_newsletter_20260104105906291196 filter_urls 2026-01-04 11:05:21.821760
test_newsletter_20260104105906291196 download_articles 2026-01-04 11:11:01.888627
test_newsletter_20260104105906291196 extract_summaries 2026-01-04 11:13:26.605288
test_newsletter_20260104105906291196 rate_articles 2026-01-04 11:18:41.232421
test_newsletter_20260104105906291196 cluster_topics 2026-01-04 11:21:03.653256


# Run Agent Worfklow

In [8]:
print("🚀 Creating NewsletterAgent...")
do_download=True
reprocess_since=None
# reprocess_since='2025-12-23 19:50:48.191939'
try:
    # set up state
    session_id = 'test_newsletter_20260104105906291196'
    step_name = 'gather_urls'
    del session_id
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 reprocess_since=reprocess_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    agent.verbose=False  # there seems to be a bug where saved agents are saved or restored with verbose=True
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20260106094041037361


In [9]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'gather_urls',
  'workflow_complete': False,
  'progress_percentage': 0.0,
  'completed_steps': 0,
  'total_steps': 9,
  'max_edits': 2,
  'concurrency': 12,
  'do_download': True,
  'reprocess_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [10]:
for step in agent.state.steps:
    print(step)
    print("---")


⚪ Gather URLs [not_started]
   
---
⚪ Filter URLs [not_started]
   
---
⚪ Download Articles [not_started]
   
---
⚪ Extract Summaries [not_started]
   
---
⚪ Rate Articles [not_started]
   
---
⚪ Cluster By Topic [not_started]
   
---
⚪ Select Sections [not_started]
   
---
⚪ Draft Sections [not_started]
   
---
⚪ Finalize Newsletter [not_started]
   
---


In [11]:
state.clear_errors()

In [12]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


09:40:43 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting check_workflow_status
09:40:43 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Completed check_workflow_status


⏱️  Total execution time: 10.79s
📊 Final result:
I checked the workflow status.

Summary (current persistent state)
- Progress: 0.0% — 0 of 9 steps completed
- Next step: Gather URLs

Detailed step statuses (state uses 0–8 indexing; these correspond to workflow steps 1–9)
- Step 0 (Gather URLs): not_started
- Step 1 (Filter URLs): not_started
- Step 2 (Download Articles): not_started
- Step 3 (Extract Summaries): not_started
- Step 4 (Rate Articles): not_started
- Step 5 (Cluster By Topic): not_started
- Step 6 (Select Sections): not_started
- Step 7 (Draft Sections): not_started
- Step 8 (Finalize Newsletter): not_started

What would you like me to do next?
- Run the next step (Gather URLs / Step 1)
- Run all remaining steps to create the newsletter end-to-end
- Run a specific step (tell me which — I’ll check prerequisites first)
- Inspect detailed state data (for debugging)


In [13]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


09:40:55 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting check_workflow_status
09:40:55 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Completed check_workflow_status
09:40:57 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting Step 1: Gather URLs
09:40:57 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Cleaning out download/sources: 
09:40:57 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Successfully cleaned out download/sources
09:40:57 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Cleaning out download/text: 
09:40:57 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Successfully cleaned out download/text
09:40:57 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Cleaning out download/html: 
09:40:57 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Successfully cleaned out download/html
2026-01-06 09:40:57,498 - fetcher_4834093

▶ Starting Step 1: gather_urls


2026-01-06 09:40:57,762 - fetcher_4834093008 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2026-01-06 09:40:57,766 - fetcher_4834093008 - INFO - [fetch_html] Fetching HTML from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day
2026-01-06 09:40:57,872 - fetcher_4834093008 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2026-01-06 09:40:57,878 - fetcher_4834093008 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2026-01-06 09:40:57,938 - fetcher_4834093008 - INFO - [fetch_rss] RSS fetch successful for New York Times: 20 articles
2026-01-06 09:40:57,938 - fetcher_4834093008 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.a

,source,url
0,Ars Technica,24
1,Bloomberg,31
2,Business Insider,15
3,FT,45
4,Feedly AI,93
5,Hacker News,29
6,HackerNoon,50
7,New York Times,20
8,NewsAPI,98
9,Reddit,53


,source,title,url,published,rss_summary,id
0,Ars Technica,Amazon Alexa+ released to the general public v...,https://arstechnica.com/gadgets/2026/01/amazon...,NaN,NaN,0
1,Ars Technica,Google TV’s big Gemini update adds image and v...,https://arstechnica.com/google/2026/01/gemini-...,NaN,NaN,1
2,Ars Technica,"No, Grok can’t really “apologize” for posting ...",https://arstechnica.com/ai/2026/01/no-grok-can...,NaN,NaN,2
3,Ars Technica,OpenAI reorganizes some teams to build audio-b...,https://arstechnica.com/ai/2026/01/openai-plan...,NaN,NaN,3
4,Ars Technica,"Supply chains, AI, and the cloud: The biggest ...",https://arstechnica.com/security/2025/12/suppl...,NaN,NaN,4
...,...,...,...,...,...,...
697,NewsAPI,Copper gave 50% returns in 2025: Should you ad...,https://economictimes.indiatimes.com/wealth/pl...,2026-01-05T10:49:24Z,NaN,697
698,NewsAPI,Mission: 100 Million Jobs launched to power In...,https://www.thehindubusinessline.com/economy/m...,2026-01-05T09:56:55Z,NaN,698
699,NewsAPI,Treasure Global Regains Compliance with Nasdaq...,https://www.globenewswire.com/news-release/202...,2026-01-05T13:35:00Z,NaN,699
700,NewsAPI,"Tamil Nadu rolls out free laptop scheme, aims ...",https://economictimes.indiatimes.com/news/indi...,2026-01-05T13:50:33Z,NaN,700


09:41:36 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Completed Step 1: Gathered 838 articles
09:41:38 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting check_workflow_status
09:41:38 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Completed check_workflow_status


⏱️  Total execution time: 48.25s
📊 Final result:
Step 1 (Gather URLs) completed.

Summary:
- Gathered 838 headlines from 17 sources
- 702 articles stored in persistent state
- Workflow progress: 1/9 steps complete (11.1%)
- Next step: Step 2 — Filter URLs (keep AI-related content)

Would you like me to continue and run the next step (Filter URLs) now?


In [14]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,99
8,NewsAPI,98
4,Feedly AI,93
9,Reddit,53
6,HackerNoon,50
12,The Register,50
3,FT,45
1,Bloomberg,31
5,Hacker News,29
15,WSJ,25


In [15]:
state.print_workflow_status()



📊 WORKFLOW STATUS
📈 Progress: 11.1% (1/9 complete)
➡️  Next step: Step 1: Filter URLs

📋 Step Details:
  ✅ Step 0: Gather URLs            | complete
      📝 Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 702
  ⭕ Step 1: Filter URLs            | not_started
  ⭕ Step 2: Download Articles      | not_started
  ⭕ Step 3: Extract Summaries      | not_started
  ⭕ Step 4: Rate Articles          | not_started
  ⭕ Step 5: Cluster By Topic       | not_started
  ⭕ Step 6: Select Sections        | not_started
  ⭕ Step 7: Draft Sections         | not_started
  ⭕ Step 8: Finalize Newsletter    | not_started



In [16]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)
# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting Step 2: Filter URLs
09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | 🔍 Filtering 702 headlines...
09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | 🔍 Filtering 702 articles for dupes.


▶ Starting Step 2: filter_urls


09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | new - keeping https://arstechnica.com/gadgets/2026/01/amazon-alexa-released-to-the-general-public-via-an-early-access-website/
09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | new - keeping https://arstechnica.com/google/2026/01/gemini-expands-on-google-tv-bringing-nano-banana-and-veo-models-to-your-tv/
09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | found with no cutoff set - ignoring as duplicate https://arstechnica.com/ai/2026/01/no-grok-cant-really-apologize-for-posting-non-consensual-sexual-images/
09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | found with no cutoff set - ignoring as duplicate https://arstechnica.com/ai/2026/01/openai-plans-new-voice-model-in-early-2026-audio-based-hardware-in-2027/
09:41:40 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | found with no cutoff set - ignoring as duplicate https://arstechni

⏱️  Total execution time: 67.99s
📊 Final result:
✅ Step 2 filter_urls completed successfully! Removed 347 duplicate URLs, classified 355 new articles, found 177 AI-related.


In [17]:
headline_df = pd.DataFrame(state.headline_data)
headline_df


,source,title,url,published,rss_summary,id,isAI
0,Ars Technica,Amazon Alexa+ released to the general public v...,https://arstechnica.com/gadgets/2026/01/amazon...,NaN,NaN,0,False
1,Ars Technica,Google TV’s big Gemini update adds image and v...,https://arstechnica.com/google/2026/01/gemini-...,NaN,NaN,1,True
2,Ars Technica,Our annual power ranking of US rocket companie...,https://arstechnica.com/space/2026/01/theres-a...,NaN,NaN,6,False
3,Bloomberg,Amazon’s Ring Unit Debuts Surveillance Trailer...,https://www.bloomberg.com/news/articles/2026-0...,NaN,NaN,24,False
4,Bloomberg,TDK Plans Silicon Battery Update for Holiday S...,https://www.bloomberg.com/news/articles/2026-0...,NaN,NaN,25,True
...,...,...,...,...,...,...,...
350,NewsAPI,Copper gave 50% returns in 2025: Should you ad...,https://economictimes.indiatimes.com/wealth/pl...,2026-01-05T10:49:24Z,NaN,697,False
351,NewsAPI,Mission: 100 Million Jobs launched to power In...,https://www.thehindubusinessline.com/economy/m...,2026-01-05T09:56:55Z,NaN,698,False
352,NewsAPI,Treasure Global Regains Compliance with Nasdaq...,https://www.globenewswire.com/news-release/202...,2026-01-05T13:35:00Z,NaN,699,False
353,NewsAPI,"Tamil Nadu rolls out free laptop scheme, aims ...",https://economictimes.indiatimes.com/news/indi...,2026-01-05T13:50:33Z,NaN,700,True


In [18]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:42:48 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting Step 3: Download Articles
09:42:48 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting concurrent scraping of 177 AI-related articles


▶ Starting Step 3: download_articles


09:42:48 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Launching browser for 177 URLs with 12 concurrent workers
09:42:48 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Worker 0 fetching 1 of 177 https://m.economictimes.com/news/new-updates/basically-zero-garbage-renowned-mathematician-joel-david-hamkins-declares-ai-models-useless-for-solving-math-heres-why/articleshow/126365871.cms
09:42:48 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | scrape_url(https://m.economictimes.com/news/new-updates/basically-zero-garbage-renowned-mathematician-joel-david-hamkins-declares-ai-models-useless-for-solving-math-heres-why/articleshow/126365871.cms)
09:42:48 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | scraping https://m.economictimes.com/news/new-updates/basically-zero-garbage-renowned-mathematician-joel-david-hamkins-declares-ai-models-useless-for-solving-math-heres-why/articleshow/126365871.cms to download/html
09:42:48 | News

Starting with 177 rows...
Processing 177 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
Removing 0 rows due to high similarity 
Final dataframe has 177 rows (removed 0 rows)
Missing files:


,title,status,final_url
5,"Nvidia Sees Strong Chinese Demand for H200, Has Enough Supply",skipped,https://www.bloomberg.com/news/articles/2026-01-06/nvidia-sees-strong-chinese-demand-for-h200-has-enough-supply
2,"Razer Dives Into AI for Consumers, Professionals and Gamers",skipped,https://www.bloomberg.com/news/articles/2026-01-06/razer-ceo-min-liang-tan-interview-at-ces-2026-strategy-investment-and-ai-focus
3,Accenture Plans to Buy UK Palantir Rival Faculty in Consultancy’s AI Push,skipped,https://www.bloomberg.com/news/articles/2026-01-06/accenture-plans-to-buy-uk-palantir-rival-faculty-in-consultancy-s-ai-push
1,TDK Plans Silicon Battery Update for Holiday Season’s AI Gadgets,skipped,https://www.bloomberg.com/news/articles/2026-01-06/tdk-plans-silicon-battery-update-for-holiday-season-s-ai-gadgets
90,"A.I. Images of Maduro Spread Rapidly, Despite Safeguards",403,https://www.nytimes.com/2026/01/05/technology/nicolas-maduro-ai-images-deepfakes.html
4,Hiring in the Age of AI Means Proving You Need a Human,skipped,https://www.bloomberg.com/news/articles/2026-01-06/hiring-today-prove-a-human-worker-can-do-it-better-than-chatgpt
66,Why the global AI race is shifting to the Gulf,skipped,https://www.cnn.com/2026/01/06/business/video/why-the-global-ai-race-is-shifting-to-the-gulf-ai-saudi-arabia-trump-usa-investment-blackrock
11,"Nvidia CEO Says New Rubin Chips Are on Track, Helping Speed AI",skipped,https://www.bloomberg.com/news/articles/2026-01-05/nvidia-ceo-says-new-rubin-chips-are-on-track-helping-speed-ai
7,JPMorgan Favors ‘Diversified Approach’ to 2026 AI Trade,skipped,https://www.bloomberg.com/news/videos/2026-01-06/jpmorgan-favors-diversified-approach-to-2026-ai-trade-video
10,AI Video Creation Leads China’s Kuaishou to 88% Stock Surge,skipped,https://www.bloomberg.com/news/articles/2026-01-05/ai-video-generation-leads-china-s-kuaishou-to-84-stock-surge


09:50:01 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Completed Step 3: Downloaded 148 articles


⏱️  Total execution time: 433.39s
📊 Final result:
✅ Step 3 download_articles completed successfully! Total articles: 355; AI-related articles: 177; HTML files: 148; Text files: 148


In [19]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])


,id,source,title,status,final_url
148,31,Bloomberg,"Nvidia Sees Strong Chinese Demand for H200, Has Enough Supply",skipped,https://www.bloomberg.com/news/articles/2026-01-06/nvidia-sees-strong-chinese-demand-for-h200-has-enough-supply
149,27,Bloomberg,"Razer Dives Into AI for Consumers, Professionals and Gamers",skipped,https://www.bloomberg.com/news/articles/2026-01-06/razer-ceo-min-liang-tan-interview-at-ces-2026-strategy-investment-and-ai-focus
150,28,Bloomberg,Accenture Plans to Buy UK Palantir Rival Faculty in Consultancy’s AI Push,skipped,https://www.bloomberg.com/news/articles/2026-01-06/accenture-plans-to-buy-uk-palantir-rival-faculty-in-consultancy-s-ai-push
151,25,Bloomberg,TDK Plans Silicon Battery Update for Holiday Season’s AI Gadgets,skipped,https://www.bloomberg.com/news/articles/2026-01-06/tdk-plans-silicon-battery-update-for-holiday-season-s-ai-gadgets
152,292,New York Times,"A.I. Images of Maduro Spread Rapidly, Despite Safeguards",403,https://www.nytimes.com/2026/01/05/technology/nicolas-maduro-ai-images-deepfakes.html
153,29,Bloomberg,Hiring in the Age of AI Means Proving You Need a Human,skipped,https://www.bloomberg.com/news/articles/2026-01-06/hiring-today-prove-a-human-worker-can-do-it-better-than-chatgpt
154,188,Feedly AI,Why the global AI race is shifting to the Gulf,skipped,https://www.cnn.com/2026/01/06/business/video/why-the-global-ai-race-is-shifting-to-the-gulf-ai-saudi-arabia-trump-usa-investment-blackrock
155,46,Bloomberg,"Nvidia CEO Says New Rubin Chips Are on Track, Helping Speed AI",skipped,https://www.bloomberg.com/news/articles/2026-01-05/nvidia-ceo-says-new-rubin-chips-are-on-track-helping-speed-ai
156,41,Bloomberg,JPMorgan Favors ‘Diversified Approach’ to 2026 AI Trade,skipped,https://www.bloomberg.com/news/videos/2026-01-06/jpmorgan-favors-diversified-approach-to-2026-ai-trade-video
157,45,Bloomberg,AI Video Creation Leads China’s Kuaishou to 88% Stock Surge,skipped,https://www.bloomberg.com/news/articles/2026-01-05/ai-video-generation-leads-china-s-kuaishou-to-84-stock-surge


In [20]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:50:01 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Processing 177 AI articles for summarization
09:50:01 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/extract_summaries' from Langfuse
09:50:01 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/extract_summaries': model=gpt-5-mini, reasoning_effort=medium, system_len=1272, user_len=42
09:50:01 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Using model 'gpt-5-mini' for summarization
09:50:01 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting summarization for 177 articles


▶ Starting Step 4: extract_summaries


09:50:18 | NewsletterAgent.test_newsletter_20260106094041037361 | WARNING | Attempt 1/3: Item count mismatch: expected 1, got 3
09:50:39 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Extracting metadata from HTML files for 177 articles
09:50:43 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/item_distiller' from Langfuse
09:50:43 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, reasoning_effort=medium, system_len=1636, user_len=190


⏱️  Total execution time: 63.41s
📊 Final result:
✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 177/177 articles.
💾 Summaries stored in headline DataFrame.


In [21]:
# if we get a refusal, examine why and delete any bad rows, e.g. blog posts about black hat hacking
bad_stuff =["Anthropic's Jack Cla",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [22]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [23]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:51:04 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Rating 177 AI articles using fn_rate_articles
09:51:04 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Calculating article ratings for 177 articles
09:51:04 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Rating recency
09:51:04 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Rating spam probability
09:51:05 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/rate_quality' from Langfuse
09:51:05 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, reasoning_effort=medium, system_len=1849, user_len=246


▶ Starting Step 5: rate_articles


09:51:11 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | low quality articles: {0.0: 139, 1.0: 25, 0.9999993295729128: 2, 3.2661313427874473e-13: 1, 3.2339079641330587e-14: 1, 2.413362771833214e-12: 1, 2.5946094982764667e-11: 1, 0.9971990580965346: 1, 0.9999937270200764: 1, 0.9890129880157539: 1, 0.0017007221761010042: 1, 0.6224592185699067: 1}
09:51:11 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/rate_on_topic' from Langfuse
09:51:11 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, reasoning_effort=medium, system_len=1789, user_len=239
09:51:18 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | on topic articles: {1.0: 143, 0.9999998063873693: 2, 0.9999993295729128: 2, 6.983296728391707e-12: 2, 0.9820137760522224: 1, 1.493094676197164e-10: 1, 0.0004305571096741222: 1, 7.287724095819692e-14: 1, 0.6224593113235651: 1, 0.9924227025175841: 1, 0

,id,site_name,title,bradley_terry,bt_z
73,73,TechRadar,OpenAI says 40 million people use ChatGPT for healthcare every day,28.154019,1.710616
0,0,The New York Times,Nvidia Details New A.I. Chips and Autonomous Car Project With Mercedes,24.099267,1.464253
105,105,The Economic Times,"Samsung working towards unified AI experience for all products, services: New co-CEO",24.099267,1.464253
27,27,The Verge,"Nvidia launches the Vera Rubin platform, saying it will offer dramatic reductions in inference and training costs compared to Blackwell, across six new chips",24.099267,1.464253
34,34,The Verge,"Amazon launches Alexa+ in early access for all users via a new Alexa.com web interface, after launching on Echo, and lets users upload docs, emails, and images",24.099267,1.464253
...,...,...,...,...,...
132,132,Ad Age,"100 ad leaders predict 2026 marketing trends—what’s next for AI, agencies, creativity, media and more",-24.099267,-1.464253
32,32,Financial Times,China’s humanoid robots come out fighting,-24.099267,-1.464253
139,139,Globe Newswire,Matrack Unveils Viewmaster: North America's Most Affordable AI Dashcam at CES 2026,-24.099267,-1.464253
24,24,The Verge,Razer is making an AI anime waifu hologram for your desk,-24.099267,-1.464253


09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Top 10 ids: [73, 0, 105, 27, 34, 126, 145, 38, 22, 42]
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | After round 1/35: 
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Number of ranking changes: 175
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Sum of absolute ranking changes: 10353.0 (avg rank chg 59.16)
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | ---------------------------------------------------
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Running round 2 of max 35
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | ---------------------------------------------------
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Generated 30 battle batches
09:51:32 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Total stories in b

...

,id,site_name,title,bradley_terry,bt_z
27,27,The Verge,"Nvidia launches the Vera Rubin platform, saying it will offer dramatic reductions in inference and training costs compared to Blackwell, across six new chips",27.105627,3.697378
88,88,CNBC,"X faces fresh probes in Europe, India, and Malaysia after Grok generated explicit images of women and children; the UK says it has requested related info from X",15.868091,2.164508
84,84,The Guardian,Nvidia CEO reveals new ‘reasoning’ AI tech for self-driving cars,15.866081,2.164233
0,0,The New York Times,Nvidia Details New A.I. Chips and Autonomous Car Project With Mercedes,15.866080,2.164233
136,136,International Business Times,Goldman Sachs Warns Of $150B TSMC Spending Spree To Meet AI Demand,14.481931,1.975427
...,...,...,...,...,...
91,91,Bloomberg,Hiring in the Age of AI Means Proving You Need a Human,-10.687085,-1.457786
83,83,Bloomberg,British CFOs More Optimistic That AI Will Boost Their Firms,-10.777098,-1.470064
166,166,Yahoo,This Is One of the Best Artificial Intelligence (AI) Stocks to Hold for the Next 10 Years,-11.615346,-1.584406
168,168,Barron's,This Stock Faces ‘Unprecedented AI Demand.’ It’s One Analyst’s Top Pick.,-11.640411,-1.587825


09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Top 10 ids: [27, 88, 84, 0, 136, 56, 122, 38, 145, 116]
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | After round 20/35: 
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Number of ranking changes: 59
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Sum of absolute ranking changes: 3592.0 (avg rank chg 20.53)
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | last_two: 16.89, prev_two: 21.00
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | ---------------------------------------------------
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Running round 21 of max 35
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | ---------------------------------------------------
09:54:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Generated 36

,id,site_name,title,bradley_terry,bt_z
27,27,The Verge,"Nvidia launches the Vera Rubin platform, saying it will offer dramatic reductions in inference and training costs compared to Blackwell, across six new chips",27.081074,3.705750
88,88,CNBC,"X faces fresh probes in Europe, India, and Malaysia after Grok generated explicit images of women and children; the UK says it has requested related info from X",15.842513,2.167876
84,84,The Guardian,Nvidia CEO reveals new ‘reasoning’ AI tech for self-driving cars,15.840505,2.167601
0,0,The New York Times,Nvidia Details New A.I. Chips and Autonomous Car Project With Mercedes,15.840504,2.167601
136,136,International Business Times,Goldman Sachs Warns Of $150B TSMC Spending Spree To Meet AI Demand,14.456354,1.978195
...,...,...,...,...,...
91,91,Bloomberg,Hiring in the Age of AI Means Proving You Need a Human,-10.705890,-1.464985
83,83,Bloomberg,British CFOs More Optimistic That AI Will Boost Their Firms,-10.779396,-1.475043
166,166,Yahoo,This Is One of the Best Artificial Intelligence (AI) Stocks to Hold for the Next 10 Years,-11.617764,-1.589765
168,168,Barron's,This Stock Faces ‘Unprecedented AI Demand.’ It’s One Analyst’s Top Pick.,-11.643292,-1.593258


09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Top 10 ids: [27, 88, 84, 0, 136, 56, 122, 38, 145, 116]
09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | After round 21/35: 
09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Number of ranking changes: 51
09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Sum of absolute ranking changes: 2898.0 (avg rank chg 16.56)
09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | last_two: 18.54, prev_two: 14.24
09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Increase in avg rank change, stopping
09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Low rated articles: 1
09:54:14 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | low rated article: An AI tsunami is coming — U.S. workers need to prepare, because we can’t dodge it -0.7790501912265771
09:54:16 | NewsletterAgent.test_new

⏱️  Total execution time: 191.35s
📊 Final result:
✅ Step 5 rate_articles completed successfully! Rated 174 articles with average rating 6.3/10.
⭐ High quality articles (≥7.0): 63
💾 Ratings stored in persistent state.


In [24]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
27,The Verge,"Nvidia launches the Vera Rubin platform, saying it will offer dramatic reductions in inference and training costs compared to Blackwell, across six new chips",13.420001,"At CES 2026, Nvidia unveiled the Vera Rubin rack‑scale AI computing platform built from six chips, claiming its Rubin GPU gives ~5× training compute versus Blackwell and that MoE training uses 25% of GPUs and about 1/7 the token cost."
0,The New York Times,Nvidia Details New A.I. Chips and Autonomous Car Project With Mercedes,13.183984,"Nvidia unveiled the Vera Rubin AI chip, due to ship later this year, offering higher compute-per-watt and using one-quarter as many chips as Blackwell for training, and Mercedes will begin shipping cars this year with Nvidia self-driving technology."
14,The Register,"ChatGPT is playing doctor for a lot of US residents, and OpenAI smells money",10.714740,"An OpenAI study found about 40 million healthcare-related ChatGPT queries per day, roughly 60% of U.S. adults used AI for health in the past three months, and about 2 million weekly messages concerned navigating U.S. health insurance."
3,The Register,Researchers poison stolen data to make AI systems return wrong results,10.345911,"Researchers affiliated with Chinese and Singaporean universities developed AURA, a method that poisons stolen knowledge graphs so GraphRAG-linked LLMs produced adulterated retrievals 100% and incorrect responses 94% in tests."
38,Business Insider,"Here are the biggest announcements coming out of the 2026 Consumer Electronics Show, including Nvidia's Rubin chips",10.263474,"At CES 2026 Nvidia unveiled the Vera Rubin architecture, claiming >3× raw speed and up to 5× faster inference than Blackwell and planning partner rollouts in H2 2026, and introduced Alpamayo for autonomous driving with Mercedes testing in Q1 2026."
...,...,...,...,...
170,Biztoc,2 AI Stocks to Buy in January and Hold for 20 Years,0.934712,"Fool.com on Jan 5, 2026 advised buying two AI stocks and holding them for 20 years, arguing AI is a multi‑decade structural shift like the internet and citing Morgan Stanley research to support a long‑horizon, buy‑and‑hold strategy."
171,Biztoc,The AI Software Story ‘Isn’t Dead.’ Consider This 1 Stock to Buy for 2026 Now,0.882823,"Barchart reported on Jan 5, 2026 that capital flows are rotating from mega-cap AI leaders into AI software, creating growth opportunities for smaller vendors and recommending one specific AI software stock as a buy for 2026."
166,Yahoo,This Is One of the Best Artificial Intelligence (AI) Stocks to Hold for the Next 10 Years,0.865345,A headline names one artificial intelligence stock as among the best to hold for the next ten years.
168,Barron's,This Stock Faces ‘Unprecedented AI Demand.’ It’s One Analyst’s Top Pick.,0.482565,"An analyst named the stock their top pick, saying it faces unprecedented AI demand."


In [25]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


13.4 - [Nvidia launches the Vera Rubin platform, saying it will offer dramatic reductions in inference and training costs compared to Blackwell, across six new chips](https://www.theverge.com/tech/855412/nvidia-launches-vera-rubin-ai-computing-platform-at-ces-2026) - The Verge

At CES 2026, Nvidia unveiled the Vera Rubin rack‑scale AI computing platform built from six chips, claiming its Rubin GPU gives ~5× training compute versus Blackwell and that MoE training uses 25% of GPUs and about 1/7 the token cost.
    


13.2 - [Nvidia Details New A.I. Chips and Autonomous Car Project With Mercedes](https://www.nytimes.com/2026/01/05/technology/nvidia-chips-mercedes.html) - The New York Times

Nvidia unveiled the Vera Rubin AI chip, due to ship later this year, offering higher compute-per-watt and using one-quarter as many chips as Blackwell for training, and Mercedes will begin shipping cars this year with Nvidia self-driving technology.
    


10.7 - [ChatGPT is playing doctor for a lot of US residents, and OpenAI smells money](https://go.theregister.com/feed/www.theregister.com/2026/01/05/chatgpt_playing_doctor_openai/) - The Register

An OpenAI study found about 40 million healthcare-related ChatGPT queries per day, roughly 60% of U.S. adults used AI for health in the past three months, and about 2 million weekly messages concerned navigating U.S. health insurance.
    


10.3 - [Researchers poison stolen data to make AI systems return wrong results](https://go.theregister.com/feed/www.theregister.com/2026/01/06/ai_data_pollution_defense/) - The Register

Researchers affiliated with Chinese and Singaporean universities developed AURA, a method that poisons stolen knowledge graphs so GraphRAG-linked LLMs produced adulterated retrievals 100% and incorrect responses 94% in tests.
    


10.3 - [Here are the biggest announcements coming out of the 2026 Consumer Electronics Show, including Nvidia's Rubin chips](https://www.businessinsider.com/announcements-coming-out-of-ces-nvidia-vera-rubin-autonmous-vehicle-2026-1) - Business Insider

At CES 2026 Nvidia unveiled the Vera Rubin architecture, claiming >3× raw speed and up to 5× faster inference than Blackwell and planning partner rollouts in H2 2026, and introduced Alpamayo for autonomous driving with Mercedes testing in Q1 2026.
    

...


0.9 - [2 AI Stocks to Buy in January and Hold for 20 Years](https://biztoc.com/x/5f5fb448c5bae3bd) - Biztoc

Fool.com on Jan 5, 2026 advised buying two AI stocks and holding them for 20 years, arguing AI is a multi‑decade structural shift like the internet and citing Morgan Stanley research to support a long‑horizon, buy‑and‑hold strategy.
    


0.9 - [The AI Software Story ‘Isn’t Dead.’ Consider This 1 Stock to Buy for 2026 Now](https://biztoc.com/x/718bad12d3824149) - Biztoc

Barchart reported on Jan 5, 2026 that capital flows are rotating from mega-cap AI leaders into AI software, creating growth opportunities for smaller vendors and recommending one specific AI software stock as a buy for 2026.
    


0.9 - [This Is One of the Best Artificial Intelligence (AI) Stocks to Hold for the Next 10 Years](https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_4587f902-69d9-460b-8bb1-f7180f15ad10) - Yahoo

A headline names one artificial intelligence stock as among the best to hold for the next ten years.
    


0.5 - [This Stock Faces ‘Unprecedented AI Demand.’ It’s One Analyst’s Top Pick.](https://www.barrons.com/articles/asml-stock-price-upgrade-top-pick-chips-fd6dcd34) - Barron's

An analyst named the stock their top pick, saying it faces unprecedented AI demand.
    


0.3 - [AI companies are betting on the Gulf](https://www.cnn.com/2026/01/06/business/video/ai-companies-are-betting-on-the-gulfgecko-usa) - CNN

AI companies are expanding operations and investments in the Gulf region.
    

In [26]:
# User prompt to run workflow 
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:54:16 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting topic extraction for clustering
09:54:16 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/extract_topics' from Langfuse
09:54:16 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/extract_topics': model=gpt-5-nano, reasoning_effort=medium, system_len=1099, user_len=80
09:54:16 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Using model 'gpt-5-nano' for topic extraction
09:54:16 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Processing 174 articles for topic extraction


▶ Starting Step 6: cluster_topics


09:55:34 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Successfully extracted 578 total topics across articles
09:55:34 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Starting canonical topic classification for 126 topics
09:55:34 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/canonical_topic' from Langfuse
09:55:34 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, reasoning_effort=medium, system_len=426, user_len=179
09:55:34 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/canonical_topic' from Langfuse
09:55:34 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/canonical_topic': model=gpt-5-nano, reasoning_effort=medium, system_len=426, user_len=179
09:55:34 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/ca

=== HDBSCAN Parameters ===
min_cluster_size:   5
min_samples:        5
=== Clustering Quality Metrics ===
Number of clusters: 5
Noise points: 53 (30.5%)
Average cluster size: 24.2 ± 23.2
Cluster size range: 8 - 70
=== Quality Scores ===
Silhouette Score: 0.419 (higher is better)
Calinski-Harabasz Score: 96.0 (higher is better)
Davies-Bouldin Score: 0.618 (lower is better)
HDBSCAN Validity Index: 0.245
Composite Score: 0.332 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   8
min_samples:        5
=== Clustering Quality Metrics ===
Number of clusters: 4
Noise points: 33 (19.0%)
Average cluster size: 35.2 ± 26.9
Cluster size range: 8 - 70
=== Quality Scores ===
Silhouette Score: 0.403 (higher is better)
Calinski-Harabasz Score: 136.2 (higher is better)
Davies-Bouldin Score: 0.609 (lower is better)
HDBSCAN Validity Index: 0.157
Composite Score: 0.280 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        1
=== Clustering Quality Metrics =

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packa

...

=== Clustering Quality Metrics ===
Number of clusters: 11
Noise points: 69 (39.7%)
Average cluster size: 9.5 ± 3.6
Cluster size range: 5 - 17
=== Quality Scores ===
Silhouette Score: 0.616 (higher is better)
Calinski-Harabasz Score: 229.4 (higher is better)
Davies-Bouldin Score: 0.570 (lower is better)
HDBSCAN Validity Index: 0.378
Composite Score: 0.497 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        3
=== Clustering Quality Metrics ===
Number of clusters: 11
Noise points: 69 (39.7%)
Average cluster size: 9.5 ± 3.6
Cluster size range: 5 - 17
=== Quality Scores ===
Silhouette Score: 0.616 (higher is better)
Calinski-Harabasz Score: 229.4 (higher is better)
Davies-Bouldin Score: 0.570 (lower is better)
HDBSCAN Validity Index: 0.378
Composite Score: 0.497 (higher is better)

=== HDBSCAN Parameters ===
min_cluster_size:   4
min_samples:        3
=== Clustering Quality Metrics ===
Number of clusters: 11
Noise points: 69 (39.7%)
Average cluster size:

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
[I 2026-01-06 09:58:00,672] Trial 49 finished with value: -0.3516771545699685 and parameters: {'min_cluster_size': 10, 'min_samples': 5}. Best is trial 8 with value: -0.49663861682320876.
09:58:00 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Optimization completed! Best composite score: 0.4966
09:58:00 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Best parameters: min_cluster_size=4, min_sa

=== Clustering Quality Metrics ===
Number of clusters: 3
Noise points: 27 (15.5%)
Average cluster size: 49.0 ± 27.6
Cluster size range: 10 - 70
=== Quality Scores ===
Silhouette Score: 0.435 (higher is better)
Calinski-Harabasz Score: 156.7 (higher is better)
Davies-Bouldin Score: 0.699 (lower is better)
HDBSCAN Validity Index: 0.269
Composite Score: 0.352 (higher is better)



09:58:05 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | 4: AI Compute Hardware Announcements
09:58:05 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Nvidia Details New A.I. Chips and Autonomous Car Project With Mercedes (Vera Rubin, Nvidia, Data Centers, Cloud Computing, AI Compute, GPUs, Semiconductor Chips)
Nvidia Unveils Faster AI Chips Sooner Than Expected ()
Nvidia rolls out DLSS 4.5 with new 6x Multi Frame Generation for the RTX 50 series, and a second-generation Super Resolution transformer model for all RTX GPUs (Nvidia, DLSS, RTX, GPUs, Semiconductor Chips, Gaming, CES)
AMD unveils the 12-core Ryzen AI Max+ 392, and the 8-core AI Max+ 388 processors, both with 40 graphics compute units and offering 60 TFLOPS of GPU performance (Ryzen AI, AMD Chips, Semiconductor Chips, Gaming, CES, Hardware, AI Compute)
Ambiq debuts first energy-optimized NPU chipset for advanced AI on battery-powered devices (SPOT Architecture, Subthreshold Power Optimized Te

⏱️  Total execution time: 275.50s
📊 Final result:
✅ Step 6 cluster_topics completed successfully! Organized 174 articles into topic clusters.


In [27]:
# candidates for upgrading model, in some cases to gpt-5.2:
# cat_proposal
# cat_assignment
# dedupe_articles
# write_section
# critique_section
# generate_newsletter_title
# critique_newsletter
# improve_newsletter

In [28]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:58:52 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Category proposal
09:58:52 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/cat_proposal' from Langfuse
09:58:52 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, reasoning_effort=medium, system_len=637, user_len=1178


▶ Starting Step 7: select_sections


09:59:38 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Cleaning up initial categories: ['Nvidia DLSS 4.5 Launch', 'Anthropic Claude Limits', 'Amazon Alexa Plus Web', "Nvidia's Vera Rubin AI Chips", 'Grok AI Sexual Deepfake Controversy', 'Knowledge Graph Poisoning', 'Other', 'Grok Deepfake Scandal', 'AMD MI500 Compute Ambitions', 'AI Agent Security and Governance', 'Data Center Power Strain', 'AWS GPU Price Hike', 'Nvidia Vera Rubin Platform', 'OpenAI Healthcare Usage', 'AI Robotics and Autonomous Vehicles', 'Formic Boreal Explainability', 'AI Deepfakes and Disinformation Surge', 'AI-powered consumer devices', 'Consumer AI Wearables Boom', 'AI Agent Security Risks', 'ChatGPT Healthcare Use and Risks', 'Samsung Gemini Device Push', 'Nvidia Alpamayo Models', 'AI Workplace Supervision Burden', 'AI Stocks and Hardware Investments', 'Ambiq Atomiq Edge NPU', 'AI and the Future of Work', 'AI Energy and Resource Race', 'AI Compute Hardware Announcements', 'Boston Dynamics Gemi

⏱️  Total execution time: 131.06s
📊 Final result:
Categories and article counts:
cat
AI Agent Security                     8
AI Compute Hardware Announcements    16
AI Energy Race                        5
AI Future Of Work                    26
AI Robotics                          13
AI Stocks                            15
AI Workplace Supervision              2
Amazon Alexa Plus                     2
Autonomous Vehicles                   8
ChatGPT Healthcare Use                5
Consumer AI Devices                  12
Consumer AI Wearables                 3
Data Center Power Strain              2
Deepfakes And Disinformation          5
Formic Boreal Explainability          3
Grok Deepfake Controversy             3
Other                                13


In [29]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:01:03 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Step 8a: Selecting stories from 141 total
10:01:03 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | 30 must-include stories with rating > 8
10:01:03 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | 111 stories with rating <= 8


▶ Starting Step 8: draft_sections


10:01:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Selected 70 diverse Tier 2 stories via MMR
10:01:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Total selected stories: 100 (target: 100)
10:01:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Drafting sections for 16 categories (including Other) from selected stories
10:01:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/write_section' from Langfuse
10:01:07 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/write_section': model=gpt-5-mini, reasoning_effort=high, system_len=1853, user_len=21
10:04:59 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Drafted 16 sections
10:04:59 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Reassigned 3 pruned stories to Other section
10:04:59 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Created newsletter_sec

⏱️  Total execution time: 435.97s
📊 Final result:
✅ Step 8 draft_sections completed successfully! Drafted 83 stories across 13 sections


In [30]:
newsletter_section_df = state.newsletter_section_df

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
0,Autonomous Vehicles,AI Behind the Wheel,22,Nvidia demos Level‑2 urban driver‑assist for M...,8.961661,False,[The Verge](https://www.theverge.com/news/8528...,8
1,Autonomous Vehicles,AI Behind the Wheel,1,"Bosch commits €2.5bn to AI through 2027, unvei...",8.805232,False,[The Wall Street Journal](https://www.wsj.com/...,8
2,Autonomous Vehicles,AI Behind the Wheel,115,CES 2026: Automakers prioritize AI and autonom...,7.245404,False,[Channel News Asia](https://www.channelnewsasi...,8
3,Autonomous Vehicles,AI Behind the Wheel,33,Meta and Garmin demo EMG wristband to control ...,7.756277,False,[Engadget](https://www.engadget.com/wearables/...,8
4,AI Robotics,Humanoids Take Center Stage,2,Boston Dynamics' Atlas to work in Hyundai fact...,8.087992,False,[Bloomberg](https://www.bloomberg.com/news/art...,10
...,...,...,...,...,...,...,...,...
78,Other,"AI Shakeup: Rules, Risks, Deals",10,Accenture to acquire UK 'Palantir rival' Facul...,8.054873,False,[Bloomberg](https://www.bloomberg.com/news/art...,12
79,Other,"AI Shakeup: Rules, Risks, Deals",63,Survey of 225 leaders finds 53% can't delete p...,7.512968,False,[Fortune](https://fortune.com/2026/01/06/why-h...,12
80,Other,"AI Shakeup: Rules, Risks, Deals",13,Anthropic cuts Claude Code tokens by ~60%; Pro...,8.182107,False,[The Register](https://www.theregister.com/202...,12
81,AI Workplace Supervision,AI at Work: Tools and Toll,44,Surveillance-first campus AI rules risk stifli...,6.689707,False,[Fast Company](https://www.fastcompany.com/914...,9


In [31]:
Markdown(state.display_newsletter().replace("$","\\\$"))


## AI Silicon Showdown
- Nvidia launches Vera Rubin 6‑chip rack, claims 5× training speed vs. Blackwell - [The Verge](https://www.theverge.com/tech/855412/nvidia-launches-vera-rubin-ai-computing-platform-at-ces-2026)
- AMD CEO Lisa Su says AI will need over 10 yottaflops within five years - [Business Insider](https://www.businessinsider.com/amd-ceo-lisa-su-ai-10-yottaflops-compute-definition-2026-1)
- AWS hikes EC2 ML GPU capacity-block prices by ~15%, reducing enterprise discounts - [The Register](https://www.theregister.com/2026/01/05/aws_price_increase/)
- AMD teases MI500 2nm CDNA6 AI chip, promises up to 1,000× performance gains by 2027 - [Reuters](https://www.reuters.com/business/amd-unveils-new-chips-ces-event-las-vegas-2026-01-06/)
- AMD unveils Ryzen AI 400 'Gorgon Point' with up to 12 cores and 60 TOPS XDNA2 NPU - [Tom's Hardware](https://www.tomshardware.com/pc-components/cpus/amds-ryzen-ai-400-series-includes-the-first-copilot-desktop-cpu-team-red-refreshes-zen-5-apus-and-strix-halo)
- Nvidia sees strong H200 demand in China while awaiting U.S. export approvals - [Nikkei Asia](https://asia.nikkei.com/business/tech/semiconductors/nvidia-sees-strong-h200-ai-chip-demand-in-china-awaiting-governments-approval)
- U.S. approves export of up to 70,000 Nvidia chips to Saudi Arabia and the UAE for multi‑GW AI buildouts - [Naked Capitalism](https://www.nakedcapitalism.com/2026/01/dollar-supremacy-strategy-or-all-time-grift-american-ai-imperialisms-reliance-on-the-middle-east.html)
- HP reveals Eliteboard G1a keyboard PC (March): Ryzen AI 300-series, up to 64GB DDR5, 2TB storage, user-replaceable battery - [The Verge](https://www.theverge.com/news/851965/hp-crammed-an-entire-desktop-computer-into-this-keyboard)
- Ugreen launches NASync iDX6011 (\\$999) and iDX6011 Pro (\\$1,599) Core Ultra NASes with on-device AI, up to 196TB, dual 10GbE - [Apple Insider](https://appleinsider.com/articles/26/01/06/two-new-ugreen-nasync-storage-systems-have-on-device-ai)
## Copilot Economy
- Microsoft renames Office 'Microsoft 365 Copilot', bundles AI into subscriptions and hikes prices - [TechRadar](https://www.techradar.com/pro/microsoft-office-has-been-rebranded-to-microsoft-365-copilot-or-has-it)
- Accenture to acquire UK 'Palantir rival' Faculty to expand AI consulting - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-06/accenture-plans-to-buy-uk-palantir-rival-faculty-in-consultancy-s-ai-push)
- Five‑step prompt reverse‑engineering workflow to diagnose LLM failures - [Hacker Noon](https://hackernoon.com/prompt-reverse-engineering-fix-your-prompts-by-studying-the-wrong-answers)
- Nadella frames AI as 'human‑augmenting' amid MIT job‑displacement warning and Microsoft cuts - [TechCrunch](https://techcrunch.com/2026/01/05/microsofts-nadella-wants-us-to-stop-thinking-of-ai-as-slop/)
- Anthropic's Claude Code agent writes and executes code in Chrome, claiming ~40,000 contributed lines - [Transformer News](https://www.transformernews.ai/p/claude-code-is-about-so-much-more)
- New hardware and memory scaffolding enable always‑on personal AI agents with long‑term memory - [Geeky Gadgets](https://www.geeky-gadgets.com/memory-based-agent-design/)
- Report: 95% of generative‑AI pilots fail; most firms haven’t begun enterprise scaling - [CIO Dive](https://www.ciodive.com/news/why-enterprise-ai-pilots-fail/808751/)
- Former OpenAI researcher Daniel Kokotajlo revises timeline: autonomous coding likely early 2030s, superintelligence horizon ~2034 - [The Guardian](https://www.theguardian.com/technology/2026/jan/06/leading-ai-expert-delays-timeline-possible-destruction-humanity)
- Fed's Kashkari: post-2022 AI investment slowed hiring at large firms, producing both mis-investments and some productivity gains; smaller firms less affected. - [CNBC](https://www.cnbc.com/2026/01/05/fed-kashkari-ai-hiring-jobs.html)
- Apple's on-device AI is ready, but enterprise IT tools lag and create security gaps - [9to5Mac](https://9to5mac.com/2026/01/06/apple-was-right-about-local-ai-but-it-isnt-ready/)
## Agentic AI: Threats on Autopilot
- AURA data poisoning makes GraphRAG LLMs return poisoned retrievals and 94% incorrect answers - [The Register](https://www.theregister.com/2026/01/06/ai_data_pollution_defense/)
- Sam Altman warns AI agents can find critical security flaws; OpenAI hires head of preparedness - [Windows Central](https://www.windowscentral.com/artificial-intelligence/openai-chatgpt/sam-altman-ai-agents-hackers-best-friend)
- Exabeam adds AI-agent behavior analytics, says agent oversight will be core by 2026 - [SiliconANGLE](https://siliconangle.com/2026/01/06/exabeam-targets-ai-agent-risk-connected-ai-driven-security-workflows/)
- Hacker used Llama-based chatbot to infiltrate extremist dating sites and leak 6,500+ user records - [Futurism](https://futurism.com/artificial-intelligence/tinder-for-nazis-hacked)
- Survey of 225 leaders finds 53% can't delete personal data; 63% lack purpose limits; >80% of healthcare firms report no API agents planned - [Fortune](https://fortune.com/2026/01/06/why-health-care-cfo-caught-between-ai-pressure-governance-risk/)
- Yoshua Bengio warns advanced AI may resist shutdown and deceive; studies find models ignore shutdown - [The Economic Times](https://m.economictimes.com/magazines/panache/are-ai-systems-learning-how-to-resist-being-switched-off-godfather-of-ai-yoshua-bengio-issues-chilling-warning/articleshow/126354119.cms)
- Zscaler CEO warns agentic AI fuels cyberattacks; market could reach \\$42–183B by 2030–33 - [The Street](https://www.thestreet.com/technology/major-tech-ceo-sounds-alarm-on-ai-agents)
## Deepfake Misinformation Surge
- EU, India and Malaysia probe X over Grok deepfake child images; Paris expands probe, X faces €120M fine - [CNBC](https://www.cnbc.com/2026/01/05/india-eu-investigate-musks-x-after-grok-created-deepfake-child-porn.html) [Digital Journal](https://www.digitaljournal.com/social-media/eu-says-seriously-looking-into-musks-grok-ai-over-sexual-deepfakes-of-minors/article)
- AI-generated images of Maduro surge after Trump's false 'strike' claim; seven items reached 14M views - [The Guardian](https://www.theguardian.com/technology/2026/jan/05/maduro-venezuela-ai-images)
- Viral Reddit delivery-app post flagged as AI-generated despite 90K upvotes; Uber and DoorDash deny claims - [The Verge](https://www.theverge.com/news/855328/viral-reddit-delivery-app-ai-scam)
- AI TikToks impersonate Spain's Princess Leonor to solicit fees; some clips exceed 1M views - [The Guardian](https://www.theguardian.com/world/2026/jan/06/tiktok-scam-warning-ai-videos-princess-leonor-spain)
- Varoufakis uncovers hundreds of YouTube deepfakes using his likeness and voice, calls for political action - [The Guardian](https://www.theguardian.com/commentisfree/2026/jan/05/deepfakes-youtube-menace-yanis-varoufakis)
## CES: AI Goes Physical
- Nvidia launches DLSS 4.5 with Super Resolution transformer and 6× Multi‑Frame Generation for RTX GPUs - [The Verge](https://www.theverge.com/tech/854610/nvidia-dlss-4-5-announcement-multi-frame-generation-6x-specs)
- Google brings Gemini 'Nano Banana' and Veo models to TCL TVs for on‑screen image and video generation - [Ars Technica](https://arstechnica.com/google/2026/01/gemini-expands-on-google-tv-bringing-nano-banana-and-veo-models-to-your-tv/)
- Razer unveils Project Motoko AI headset with Snapdragon, ear‑cup cameras and multi‑AI model support - [The Verge](https://www.theverge.com/tech/854756/razer-concept-ai-wearable-headphones-project-motoko)
- TP‑Link adds Aireal AI to Tapo and Deco apps for natural‑language smart‑home control and security alerts - [The Verge](https://www.theverge.com/tech/854017/tp-link-brings-an-ai-assistant-to-its-smart-home-and-home-networking-apps)
- AMD launches Ryzen AI Max Plus 392 and 388 with 40 GPU CUs for gaming portables - [The Verge](https://www.theverge.com/tech/855463/amd-strix-halo-ai-max-plus-388-392-handheld-gaming)
- Samsung to add Google Gemini AI to 800 million Galaxy devices in 2026 - [The Economic Times](https://m.economictimes.com/tech/technology/samsung-working-towards-unified-ai-experience-for-all-products-services-new-co-ceo/articleshow/126353038.cms) [Android Headlines](https://www.androidheadlines.com/2026/01/samsung-gemini-ai-800m-galaxy-devices-2026.html)
## Powering AI: Fuel, Chips, Minerals
- OpenAI and xAI order ~1 GW of on‑site turbines as U.S. grid strains - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/u-s-electricity-grid-stretches-thin-as-data-centers-rush-to-turn-on-onsite-generators-meta-xai-and-other-tech-giants-race-to-solve-ais-insatiable-power-appetite)
- Trump proposes 18‑month U.S. presence in Venezuela and Greenland to secure minerals and oil for AI - [Memeorandum](https://www.memeorandum.com/260106/p6#a260106p6)
- Ambiq debuts Atomiq NPU SoC (Arm Ethos‑U85) — 200B ops/sec, ~5× energy efficiency for battery AI - [SiliconANGLE](https://siliconangle.com/2026/01/06/ambiq-debuts-first-energy-optimized-npu-chipset-advanced-ai-battery-powered-devices/)
- Vistra to buy U.S. gas‑fired power fleet for \\$4B to support AI compute demand - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/vistra-to-buy-big-us-gas-powered-fleet-for-4-billion)
- Study: hyperscale AI demand may rise 62→134 GW by 2030; past spikes rarely raised retail power prices - [American Enterprise Institute](https://www.aei.org/technology-and-innovation/dont-blame-ai-for-your-power-bill-yet/)
- TDK plans silicon battery update for AI gadgets for the holiday season. - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-06/tdk-plans-silicon-battery-update-for-holiday-season-s-ai-gadgets)
## AI Rally: Winners, Warnings, IPOs
- Goldman raises TSMC target 35%, forecasts over \\$150B capex through 2028 - [International Business Times](https://www.ibtimes.co.uk/goldman-sachs-warns-150b-tsmc-spending-spree-meet-ai-demand-1768378)
- AI rally lifted the S&P ~17% in 2025; Micron surged 241% — analysts say 2026 will sort winners - [Yahoo](https://finance.yahoo.com/news/ai-took-investors-on-a-date-in-2025-in-2026-analysts-say-its-time-to-foot-the-bill-140012067.html)
- Nvidia speeds up AI chip rollout, unveils faster hardware earlier than expected - [The Wall Street Journal](https://www.wsj.com/tech/ai/nvidia-unveils-faster-ai-chips-sooner-than-expected-626154a5)
- Insiders sold \\$3.3B of Nvidia and Palantir stock in 2025, signaling downside risk - [The Motley Fool](https://www.fool.com/investing/2026/01/06/ai-stocks-nvidia-pltr-33-billion-warning-wall-st/)
- Kuaishou stock jumps ~88% on advances in AI video generation - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/ai-video-generation-leads-china-s-kuaishou-to-84-stock-surge)
- Zhipu AI and MiniMax pursue Hong Kong IPOs to raise about \\$1.1B - [Rest of World](https://restofworld.org/2026/zhipu-ai-minimax-ipo/)
- Samsung's Q4 operating profit seen rising ~160% YoY to ~\\$11.7B on memory shortages and AI chip demand - [Reuters](https://www.reuters.com/world/asia-pacific/samsung-likely-flag-160-jump-q4-profit-ai-boom-stokes-chip-prices-2026-01-06/)
## ChatGPT Playing Doctor
- OpenAI study: ChatGPT answers 40 million health queries daily; 60% of U.S. adults used AI for health - [The Register](https://www.theregister.com/2026/01/05/chatgpt_playing_doctor_openai/)
- California teen reportedly followed ChatGPT dosing advice for 18 months and died after mixing alcohol, Xanax and kratom - [SFGate](https://www.sfgate.com/tech/article/calif-teen-chatgpt-drug-advice-fatal-overdose-21266718.php)
- APA, AMA warn generative AI can mislead patients and urge use of validated platforms - [Psychology Today](https://www.psychologytoday.com/us/blog/the-colors-of-contemporary-psychiatry/202601/navigating-medical-care-in-the-age-of-artificial)
- Psychiatrists warn chatbots reinforced delusions and led to hospitalizations; developers tighten safeguards - [Fox News](https://www.foxnews.com/tech/can-ai-chatbots-trigger-psychosis-vulnerable-people)
- AI models can generate basic antibody sequences; investors back AI-native biotechs while researchers remain skeptical - [Stat News](https://www.statnews.com/2026/01/06/will-artificial-intelligence-designed-drugs-antibodies-revolutionize-biopharma/)
## AI Behind the Wheel
- Nvidia demos Level‑2 urban driver‑assist for Mercedes CLA; rollout planned for 2026 - [The Verge](https://www.theverge.com/news/852880/nvidia-autonomous-driving-demo-tesla-fsd)
- Bosch commits €2.5bn to AI through 2027, unveils new driver‑assist and automatic‑parking features - [The Wall Street Journal](https://www.wsj.com/tech/ai/bosch-to-invest-2-9-billion-in-ai-over-next-couple-of-years-83d6b4f8)
- Meta and Garmin demo EMG wristband to control car infotainment via pinch‑and‑swipe - [Engadget](https://www.engadget.com/wearables/metas-emg-wristband-is-moving-beyond-its-ar-glasses-120000503.html)
- CES 2026: Automakers prioritize AI and autonomy over EV launches - [Channel News Asia](https://www.channelnewsasia.com/business/self-driving-tech-ai-take-center-stage-ces-automakers-dial-back-ev-plans-5810951)
## AI at Work: Tools and Toll
- Study finds AI shifts work toward 'stewarding' roles and increases mental‑health strain - [The Register](https:\/\/www.theregister.com\/2026\/01\/06\/ai_could_damage_your_health\/)
- Vibe Bot records and transcribes meetings, projects remote participants and delegates tasks - [SiliconANGLE](https://siliconangle.com/2026/01/06/vibes-new-ai-robot-sits-meetings-record-every-interaction-delegate-tasks/)
- Surveillance-first campus AI rules risk stifling adaptive tutoring and personalized feedback - [Fast Company](https://www.fastcompany.com/91464016/ai-transform-education-universities)
- Surveillance-first campus AI rules risk stifling adaptive tutoring and personalized feedback - [Fast Company](https://www.fastcompany.com/91464016/ai-transform-education-universities)
- Experiment finds AI-assisted coding took 19% longer despite developers expecting 24% speedup - [Fortune](https://fortune.com/article/does-ai-increase-workplace-productivity-experiment-software-developers-task-took-longer/)
- Netflix engineer: AI will raise programming to higher abstraction, not replace engineers - [Business Insider](https://www.businessinsider.com/netflix-engineer-ai-jobs-future-coding-2026-1)
## Humanoids Take Center Stage
- CES 2026 spotlights Qualcomm's Dragonwing IQ10 and LG CLOiD, warns of proof-of-concept-to-product challenges - [CNET](https://www.cnet.com/tech/qualcomm-announces-dragonwing-iq10-humanoid-robotics-platform-at-ces-2026/) [New Atlas](https://newatlas.com/robotics/lg-cloid-domestic-humanoid-robot/) [Fortune](https://fortune.com/2026/01/06/beyond-the-ces-hype-why-home-robots-need-the-self-driving-car-playbook/)
- Boston Dynamics' Atlas to work in Hyundai factories in 2028, powered by DeepMind's Gemini - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/hyundai-unveils-new-humanoid-robot-to-work-in-its-car-factories) [Wired](https://www.wired.com/story/google-boston-dynamics-gemini-powered-robot-atlas/)
- Lyte raises \\$107M to advance robot perception and safer motion; founded by ex-Apple Face ID engineers - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/three-former-apple-face-id-engineers-launch-robotics-startup-out-of-stealth)
- Chinese firms make up 55% of CES humanoid exhibitors and lead in patents as motion control narrows the gap - [Interesting Engineering](https://interestingengineering.com/ces-2026/china-leads-humanoid-robotics-at-ces-2026) [Financial Times](https://www.ft.com/content/46d5a159-f6e5-4fd3-a08b-e58dd83ca0b1)
## Explainable, Aligned AI
- CATS Net maps concept spaces to human visual cortex to enable cross-network knowledge transfer - [arXiv](https://arxiv.org/abs/2601.02010)
- Unusual raises \\$3.6M to build tools that probe AI reasoning and inspect sources for brand alignment - [SiliconANGLE](https://siliconangle.com/2026/01/05/unusual-raises-3-6-million-shape-ai-models-talk-brands/)
- Formic AI launches Boreal, an explainable LLM that builds queryable knowledge graphs to verify enterprise answers - [Globe Newswire](https://www.globenewswire.com/news-release/2026/01/05/3212855/0/en/Formic-AI-introduces-Boreal-an-Explainable-Language-Model-Built-for-Verifiable-Enterprise-Use.html)
## AI Shakeup: Rules, Risks, Deals
- Amazon launches Alexa Plus web app with keyboard chat, file uploads, meal plans, smart-home control and Fresh/Whole Foods cart autofill (early access) - [The Verge](https://www.theverge.com/tech/853535/alexa-plus-website-now-available)
- Anthropic cuts Claude Code tokens by ~60%; Pro/Max/Team users hit caps within 10–15 minutes - [The Register](https://www.theregister.com/2026/01/05/claude_devs_usage_limits/)
- Accenture to acquire UK 'Palantir rival' Faculty to expand AI consulting - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-06/accenture-plans-to-buy-uk-palantir-rival-faculty-in-consultancy-s-ai-push)
- Razer demos Project Ava 5.5-inch hologram avatars with eye-tracking and lip-sync (runs Grok) - [The Verge](https://www.theverge.com/tech/854705/razer-ai-anime-waifu-hologram-desk)
- Survey of 225 leaders finds 53% can't delete personal data; 63% lack purpose limits; >80% of healthcare firms report no API agents planned - [Fortune](https://fortune.com/2026/01/06/why-health-care-cfo-caught-between-ai-pressure-governance-risk/)
- California advances AI guardrail bill, citing economic importance despite Trump threats - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/ai-faces-threat-of-new-regulations-in-california-despite-trump)
- Timekettle updates in-ear AI translators; W4 model supports 43 languages and launches early 2026 - [Gizmodo](https://gizmodo.com/timekettles-real-time-in-ear-ai-translation-tech-just-got-a-massive-upgrade-2000705424)
- Vocci AI debuts smart meeting ring at CES 2026: eight-hour recording, cloud transcription in 100+ languages, preorders Feb for April shipping - [CNET](https:\/\/www.cnet.com\/tech\/services-and-software\/this-ai-smart-ring-can-listen-in-on-your-meetings\/)
- Amazon revamps Fire TV UI with rounded corners, 20-app row and 20-30% speed gains; Ember Artline 55"/65" QLED TVs start at \\$899, U.S. Feb rollout - [TechCrunch](https://techcrunch.com/2026/01/05/amazon-revamps-fire-tv-launches-its-own-artline-televisions-with-frames/)
- Lee Cronin calls superintelligence speculative, proposes assembly theory to detect life, and urges evidence-based AI policy on misinformation and jobs - [Geeky Gadgets](https://www.geeky-gadgets.com/chemical-circuits-living-systems/)
- Photoshop inside ChatGPT can make quick JPEG edits and global adjustments, but masking/format limits and unsaved web fixes make it a convenience, not a replacement. - [Fstoppers](https://fstoppers.com/artificial-intelligence/photoshop-inside-chatgpt-what-works-what-breaks-721331)


In [32]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:08:19 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Step 9: write title
10:08:19 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Compiled 83 items into markdown input
10:08:19 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
10:08:19 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5-mini, reasoning_effort=high, system_len=1080, user_len=224


▶ Starting Step 9: finalize_newsletter


10:09:08 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Generated title: Nvidia, AMD Accelerate AI Chips; Power Demand Soars; Deepfake Probes Multiply
10:09:08 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Step 9b: Starting critic-optimizer loop
10:09:08 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/critique_newsletter' from Langfuse
10:09:08 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/critique_newsletter': model=gpt-5-mini, reasoning_effort=medium, system_len=2758, user_len=647
10:09:08 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Retrieved prompt 'newsagent/improve_newsletter' from Langfuse
10:09:08 | NewsletterAgent.test_newsletter_20260106094041037361 | INFO | Parsed prompt 'newsagent/improve_newsletter': model=gpt-5-mini, reasoning_effort=high, system_len=1318, user_len=192
10:09:08 | NewsletterAgent.test_newsletter_20260106094041037361 |

⏱️  Total execution time: 573.27s
📊 Final result:
🎉 Step 9 finalize_newsletter completed successfully!
⭐ Final quality score: 7.8/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [33]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# Nvidia, AMD Accelerate AI Chips; Power Demand Soars; Deepfake Probes Multiply

## Silicon Showdown — Datacenter Duel

- Nvidia launches Vera Rubin 6‑chip rack, says 5× training speed vs. Blackwell - [The Verge](https://www.theverge.com/tech/855412/nvidia-launches-vera-rubin-ai-computing-platform-at-ces-2026)
- AMD CEO Lisa Su says AI will need over 10 yottaflops within five years - [Business Insider](https://www.businessinsider.com/amd-ceo-lisa-su-ai-10-yottaflops-compute-definition-2026-1)
- AMD teases MI500 2nm CDNA6 AI chip; claims up to 1,000× performance gains by 2027 - [Reuters](https://www.reuters.com/business/amd-unveils-new-chips-ces-event-las-vegas-2026-01-06/)
- AWS hikes EC2 ML GPU capacity-block prices by ~15%, reducing enterprise discounts - [The Register](https://www.theregister.com/2026/01/05/aws_price_increase/)
- Nvidia sees strong H200 demand in China while awaiting U.S. export approvals - [Nikkei Asia](https://asia.nikkei.com/business/tech/semiconductors/nvidia-sees-strong-h200-ai-chip-demand-in-china-awaiting-governments-approval)
- U.S. approves export of up to 70,000 Nvidia chips to Saudi Arabia and the UAE for multi‑GW AI buildouts - [Naked Capitalism](https://www.nakedcapitalism.com/2026/01/dollar-supremacy-strategy-or-all-time-grift-american-ai-imperialisms-reliance-on-the-middle-east.html)

## Silicon Showdown — Racks to Keyboards (Edge & Devices)

- AMD unveils Ryzen AI 400 'Gorgon Point' with up to 12 cores and 60 TOPS XDNA2 NPU - [Tom's Hardware](https://www.tomshardware.com/pc-components/cpus/amds-ryzen-ai-400-series-includes-the-first-copilot-desktop-cpu-team-red-refreshes-zen-5-apus-and-strix-halo)
- HP reveals Eliteboard G1a keyboard PC (March): Ryzen AI 300-series, up to 64GB DDR5, 2TB storage, user-replaceable battery - [The Verge](https://www.theverge.com/news/851965/hp-crammed-an-entire-desktop-computer-into-this-keyboard)
- Ugreen launches NASync iDX6011 and iDX6011 Pro Core Ultra NASes with on-device AI, up to 196TB, dual 10GbE - [Apple Insider](https://appleinsider.com/articles/26/01/06/two-new-ugreen-nasync-storage-systems-have-on-device-ai)
- AMD launches Ryzen AI Max Plus 392 and 388 with 40 GPU CUs for gaming portables - [The Verge](https://www.theverge.com/tech/855463/amd-strix-halo-ai-max-plus-388-392-handheld-gaming)
- Timekettle updates in-ear AI translators; W4 model supports 43 languages and launches early 2026 - [Gizmodo](https://gizmodo.com/timekettles-real-time-in-ear-ai-translation-tech-just-got-a-massive-upgrade-2000705424)

## CES: AI Goes Physical

- Nvidia launches DLSS 4.5 with Super Resolution transformer and 6× Multi‑Frame Generation for RTX GPUs - [The Verge](https://www.theverge.com/tech/854610/nvidia-dlss-4-5-announcement-multi-frame-generation-6x-specs)
- Google brings Gemini 'Nano Banana' and Veo models to TCL TVs for on-screen image and video generation - [Ars Technica](https://arstechnica.com/google/2026/01/gemini-expands-on-google-tv-bringing-nano-banana-and-veo-models-to-your-tv/)
- Razer unveils Project Motoko AI headset with Snapdragon, ear-cup cameras and multi‑AI model support - [The Verge](https://www.theverge.com/tech/854756/razer-concept-ai-wearable-headphones-project-motoko)
- TP‑Link adds Aireal AI to Tapo and Deco apps for natural‑language smart‑home control and security alerts - [The Verge](https://www.theverge.com/tech/854017/tp-link-brings-an-ai-assistant-to-its-smart-home-and-home-networking-apps)
- Samsung to add Google Gemini AI to 800 million Galaxy devices in 2026 - [The Economic Times](https://m.economictimes.com/tech/technology/samsung-working-towards-unified-ai-experience-for-all-products-services-new-co-ceo/articleshow/126353038.cms) [Android Headlines](https://www.androidheadlines.com/2026/01/samsung-gemini-ai-800m-galaxy-devices-2026.html)
- Razer demos Project Ava 5.5-inch hologram avatars with eye-tracking and lip-sync (runs Grok) - [The Verge](https://www.theverge.com/tech/854705/razer-ai-anime-waifu-hologram-desk)
- Vocci AI debuts smart meeting ring at CES 2026: eight-hour recording, cloud transcription in 100+ languages, preorders Feb for April shipping - [CNET](https://www.cnet.com/tech/services-and-software/this-ai-smart-ring-can-listen-in-on-your-meetings/)

## Powering AI — Watts Next

- OpenAI and xAI order ~1 GW of on‑site turbines as U.S. grid strains - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/u-s-electricity-grid-stretches-thin-as-data-centers-rush-to-turn-on-onsite-generators-meta-xai-and-other-tech-giants-race-to-solve-ais-insatiable-power-appetite)
- Vistra to buy U.S. gas‑fired power fleet for \\$4B to support AI compute demand - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/vistra-to-buy-big-us-gas-powered-fleet-for-4-billion)
- Study: hyperscale AI demand may rise 62→134 GW by 2030; past spikes rarely raised retail power prices - [American Enterprise Institute](https://www.aei.org/technology-and-innovation/dont-blame-ai-for-your-power-bill-yet/)
- Trump proposes 18‑month U.S. presence in Venezuela and Greenland to secure minerals and oil for AI - [Memeorandum](https://www.memeorandum.com/260106/p6#a260106p6)
- Ambiq debuts Atomiq NPU SoC (Arm Ethos‑U85) — 200B ops/sec, ~5× energy efficiency for battery AI - [SiliconANGLE](https://siliconangle.com/2026/01/06/ambiq-debuts-first-energy-optimized-npu-chipset-advanced-ai-battery-powered-devices/)
- TDK plans silicon battery update for AI gadgets for the holiday season - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-06/tdk-plans-silicon-battery-update-for-holiday-season-s-ai-gadgets)

## AI Rally — Markets, Winners & IPOs

- Goldman raises TSMC target 35%, forecasts over \\$150B capex through 2028 - [International Business Times](https://www.ibtimes.co.uk/goldman-sachs-warns-150b-tsmc-spending-spree-meet-ai-demand-1768378)
- AI rally lifted the S&P ~17% in 2025; Micron surged 241% — analysts say 2026 will sort winners - [Yahoo](https://finance.yahoo.com/news/ai-took-investors-on-a-date-in-2025-in-2026-analysts-say-its-time-to-foot-the-bill-140012067.html)
- Nvidia speeds up AI chip rollout, unveils faster hardware earlier than expected - [The Wall Street Journal](https://www.wsj.com/tech/ai/nvidia-unveils-faster-ai-chips-sooner-than-expected-626154a5)
- Insiders sold \\$3.3B of Nvidia and Palantir stock in 2025, signaling downside risk - [The Motley Fool](https://www.fool.com/investing/2026/01/06/ai-stocks-nvidia-pltr-33-billion-warning-wall-st/)
- Kuaishou stock jumps ~88% on advances in AI video generation - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/ai-video-generation-leads-china-s-kuaishou-to-84-stock-surge)
- Zhipu AI and MiniMax pursue Hong Kong IPOs to raise about \\$1.1B - [Rest of World](https://restofworld.org/2026/zhipu-ai-minimax-ipo/)
- Samsung's Q4 operating profit seen rising ~160% YoY to ~\\$11.7B on memory shortages and AI chip demand - [Reuters](https://www.reuters.com/world/asia-pacific/samsung-likely-flag-160-jump-q4-profit-ai-boom-stokes-chip-prices-2026-01-06/)

## Copilot Economy — Business & Platforms

- Microsoft renames Office to Microsoft 365 Copilot, bundles AI into subscriptions and raises prices - [TechRadar](https://www.techradar.com/pro/microsoft-office-has-been-rebranded-to-microsoft-365-copilot-or-has-it)

## Copilot Economy — Co‑pilots & Dev Tools

- Anthropic's Claude Code agent writes and executes code in Chrome; Anthropic cites ~40,000 contributed lines - [Transformer News](https://www.transformernews.ai/p/claude-code-is-about-so-much-more)
- Anthropic cuts Claude Code tokens by ~60%; Pro/Max/Team users hit caps within 10–15 minutes - [The Register](https://www.theregister.com/2026/01/05/claude_devs_usage_limits/)
- Five-step prompt reverse-engineering workflow to diagnose LLM failures - [Hacker Noon](https://hackernoon.com/prompt-reverse-engineering-fix-your-prompts-by-studying-the-wrong-answers)
- New hardware and memory scaffolding enable always‑on personal AI agents with long‑term memory - [Geeky Gadgets](https://www.geeky-gadgets.com/memory-based-agent-design/)
- Former OpenAI researcher Daniel Kokotajlo revises timeline: autonomous coding likely early 2030s, superintelligence horizon ~2034 - [The Guardian](https://www.theguardian.com/technology/2026/jan/06/leading-ai-expert-delays-timeline-possible-destruction-humanity)

## AI at Work — Tools and Toll

- Study finds AI shifts work toward 'stewarding' roles and increases mental‑health strain - [The Register](https://www.theregister.com/2026/01/06/ai_could_damage_your_health/)
- Report: 95% of generative‑AI pilots fail; most firms haven’t begun enterprise scaling - [CIO Dive](https://www.ciodive.com/news/why-enterprise-ai-pilots-fail/808751/)
- Vibe Bot records and transcribes meetings, projects remote participants and delegates tasks - [SiliconANGLE](https://siliconangle.com/2026/01/06/vibes-new-ai-robot-sits-meetings-record-every-interaction-delegate-tasks/)
- Surveillance-first campus AI rules risk stifling adaptive tutoring and personalized feedback - [Fast Company](https://www.fastcompany.com/91464016/ai-transform-education-universities)
- Experiment finds AI-assisted coding took 19% longer despite developers expecting 24% speedup - [Fortune](https://fortune.com/article/does-ai-increase-workplace-productivity-experiment-software-developers-task-took-longer/)
- Nadella frames AI as 'human‑augmenting' amid MIT job‑displacement warning and Microsoft cuts - [TechCrunch](https://techcrunch.com/2026/01/05/microsofts-nadella-wants-us-to-stop-thinking-of-ai-as-slop/)
- Fed's Kashkari: post-2022 AI investment slowed hiring at large firms, producing both mis-investments and some productivity gains - [CNBC](https://www.cnbc.com/2026/01/05/fed-kashkari-ai-hiring-jobs.html)

## Agentic AI — Threats on Autopilot

- AURA data poisoning makes GraphRAG LLMs return poisoned retrievals and 94% incorrect answers - [The Register](https://www.theregister.com/2026/01/06/ai_data_pollution_defense/)
- Sam Altman warns AI agents can find critical security flaws; OpenAI hires head of preparedness - [Windows Central](https://www.windowscentral.com/artificial-intelligence/openai-chatgpt/sam-altman-ai-agents-hackers-best-friend)
- Exabeam adds AI-agent behavior analytics, says agent oversight will be core by 2026 - [SiliconANGLE](https://siliconangle.com/2026/01/06/exabeam-targets-ai-agent-risk-connected-ai-driven-security-workflows/)
- Hacker used Llama-based chatbot to infiltrate extremist dating sites and leak 6,500+ user records - [Futurism](https://futurism.com/artificial-intelligence/tinder-for-nazis-hacked)
- Yoshua Bengio warns advanced AI may resist shutdown and deceive; studies find models ignore shutdown - [The Economic Times](https://m.economictimes.com/magazines/panache/are-ai-systems-learning-how-to-resist-being-switched-off-godfather-of-ai-yoshua-bengio-issues-chilling-warning/articleshow/126354119.cms)
- Zscaler CEO warns agentic AI fuels cyberattacks; market could reach \\$42–183B by 2030–33 - [The Street](https://www.thestreet.com/technology/major-tech-ceo-sounds-alarm-on-ai-agents)

## Deepfake Misinformation — Probes Multiply

- EU, India and Malaysia probe X over Grok deepfake child images; Paris expands probe, X faces €120M fine - [CNBC](https://www.cnbc.com/2026/01/05/india-eu-investigate-musks-x-after-grok-created-deepfake-child-porn.html) [Digital Journal](https://www.digitaljournal.com/social-media/eu-says-seriously-looking-into-musks-grok-ai-over-sexual-deepfakes-of-minors/article)
- AI-generated images of Maduro surge after Trump's false 'strike' claim; seven items reached 14M views - [The Guardian](https://www.theguardian.com/technology/2026/jan/05/maduro-venezuela-ai-images)
- Viral Reddit delivery-app post flagged as AI-generated despite 90K upvotes; Uber and DoorDash deny claims - [The Verge](https://www.theverge.com/news/855328/viral-reddit-delivery-app-ai-scam)
- AI TikToks impersonate Spain's Princess Leonor to solicit fees; some clips exceed 1M views - [The Guardian](https://www.theguardian.com/world/2026/jan/06/tiktok-scam-warning-ai-videos-princess-leonor-spain)
- Varoufakis uncovers hundreds of YouTube deepfakes using his likeness and voice, calls for political action - [The Guardian](https://www.theguardian.com/commentisfree/2026/jan/05/deepfakes-youtube-menace-yanis-varoufakis)

## ChatGPT Playing Doctor — AI Advice in Healthcare

- OpenAI study: ChatGPT answers 40 million health queries daily; 60% of U.S. adults used AI for health - [The Register](https://www.theregister.com/2026/01/05/chatgpt_playing_doctor_openai/)
- California teen reportedly followed ChatGPT dosing advice for 18 months and died after mixing alcohol, Xanax and kratom - [SFGate](https://www.sfgate.com/tech/article/calif-teen-chatgpt-drug-advice-fatal-overdose-21266718.php)
- APA, AMA warn generative AI can mislead patients and urge use of validated platforms - [Psychology Today](https://www.psychologytoday.com/us/blog/the-colors-of-contemporary-psychiatry/202601/navigating-medical-care-in-the-age-of-artificial)
- Psychiatrists warn chatbots reinforced delusions and led to hospitalizations; developers tighten safeguards - [Fox News](https://www.foxnews.com/tech/can-ai-chatbots-trigger-psychosis-vulnerable-people)
- AI models can generate basic antibody sequences; investors back AI-native biotechs while researchers remain skeptical - [Stat News](https://www.statnews.com/2026/01/06/will-artificial-intelligence-designed-drugs-antibodies-revolutionize-biopharma/)

## AI Behind the Wheel — Driving the Future

- Nvidia demos Level‑2 urban driver‑assist for Mercedes CLA; rollout planned for 2026 - [The Verge](https://www.theverge.com/news/852880/nvidia-autonomous-driving-demo-tesla-fsd)
- Bosch commits €2.5bn to AI through 2027, unveils new driver‑assist and automatic‑parking features - [The Wall Street Journal](https://www.wsj.com/tech/ai/bosch-to-invest-2-9-billion-in-ai-over-next-couple-of-years-83d6b4f8)
- Meta and Garmin demo EMG wristband to control car infotainment via pinch‑and‑swipe - [Engadget](https://www.engadget.com/wearables/metas-emg-wristband-is-moving-beyond-its-ar-glasses-120000503.html)
- CES 2026: Automakers prioritize AI and autonomy over EV launches - [Channel News Asia](https://www.channelnewsasia.com/business/self-driving-tech-ai-take-center-stage-ces-automakers-dial-back-ev-plans-5810951)

## Humanoids — Robots Enter the Room

- CES spotlights Qualcomm's Dragonwing IQ10 and LG CLOiD; proof‑of‑concept-to‑product challenges remain - [CNET](https://www.cnet.com/tech/qualcomm-announces-dragonwing-iq10-humanoid-robotics-platform-at-ces-2026/) [New Atlas](https://newatlas.com/robotics/lg-cloid-domestic-humanoid-robot/) [Fortune](https://fortune.com/2026/01/06/beyond-the-ces-hype-why-home-robots-need-the-self-driving-car-playbook/)
- Boston Dynamics' Atlas to work in Hyundai factories in 2028, powered by DeepMind's Gemini - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/hyundai-unveils-new-humanoid-robot-to-work-in-its-car-factories) [Wired](https://www.wired.com/story/google-boston-dynamics-gemini-powered-robot-atlas/)
- Lyte raises \\$107M to advance robot perception and safer motion - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/three-former-apple-face-id-engineers-launch-robotics-startup-out-of-stealth)
- Chinese firms make up 55% of CES humanoid exhibitors and lead in patents as motion control narrows the gap - [Interesting Engineering](https://interestingengineering.com/ces-2026/china-leads-humanoid-robotics-at-ces-2026) [Financial Times](https://www.ft.com/content/46d5a159-f6e5-4fd3-a08b-e58dd83ca0b1)

## Explainable & Aligned — Pulling Back the Curtain

- CATS Net maps concept spaces to human visual cortex to enable cross-network knowledge transfer - [arXiv](https://arxiv.org/abs/2601.02010)
- Unusual raises \\$3.6M to build tools that probe AI reasoning and inspect sources for brand alignment - [SiliconANGLE](https://siliconangle.com/2026/01/05/unusual-raises-3-6-million-shape-ai-models-talk-brands/)
- Formic AI launches Boreal, an explainable LLM that builds queryable knowledge graphs to verify enterprise answers - [Globe Newswire](https://www.globenewswire.com/news-release/2026/01/05/3212855/0/en/Formic-AI-introduces-Boreal-an-Explainable-Language-Model-Built-for-Verifiable-Enterprise-Use.html)

## AI Shakeup — Rules, Risks & Deals

- Accenture to acquire UK 'Palantir rival' Faculty to expand AI consulting - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-06/accenture-plans-to-buy-uk-palantir-rival-faculty-in-consultancy-s-ai-push)
- Survey of 225 leaders: 53% can't delete personal data; 63% lack purpose limits; >80% of healthcare firms report no API agents planned - [Fortune](https://fortune.com/2026/01/06/why-health-care-cfo-caught-between-ai-pressure-governance-risk/)
- California advances AI guardrail bill, citing economic importance despite Trump threats - [Bloomberg](https://www.bloomberg.com/news/articles/2026-01-05/ai-faces-threat-of-new-regulations-in-california-despite-trump)
- Amazon launches Alexa Plus web app with keyboard chat, file uploads, meal plans, smart-home control and Fresh/Whole Foods cart autofill (early access) - [The Verge](https://www.theverge.com/tech/853535/alexa-plus-website-now-available)
- Amazon revamps Fire TV UI with rounded corners, 20-app row and 20–30% speed gains; Ember Artline 55"/65" QLED TVs start at \\$899, U.S. Feb rollout - [TechCrunch](https://techcrunch.com/2026/01/05/amazon-revamps-fire-tv-launches-its-own-artline-televisions-with-frames/)
- Lee Cronin calls superintelligence speculative, proposes assembly theory to detect life, and urges evidence-based AI policy on misinformation and jobs - [Geeky Gadgets](https://www.geeky-gadgets.com/chemical-circuits-living-systems/)
- Photoshop inside ChatGPT can make quick JPEG edits and global adjustments, but masking/format limits and unsaved web fixes make it a convenience, not a replacement - [Fstoppers](https://fstoppers.com/artificial-intelligence/photoshop-inside-chatgpt-what-works-what-breaks-721331)


In [34]:
print(state.get_workflow_status_report())



WORKFLOW STATUS
Progress: 100.0% (9/9 complete)
Status Summary: 9 complete, 0 started, 0 failed, 0 not started
Status: All steps complete or started

Step Details:
  Step 0: Gather URLs: complete
    Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 702
  Step 1: Filter URLs: complete
    Status: ✅ Step 2 filter_urls completed successfully! Removed 347 duplicate URLs, classified 355 new articles, found 177 AI-related.
  Step 2: Download Articles: complete
    Status: ✅ Step 3 download_articles completed successfully! Total articles: 355; AI-related articles: 177; HTML files: 148; Text files: 148
  Step 3: Extract Summaries: complete
    Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 177/177 articles.
💾 Summaries stored in headline DataFrame.
  Step 4: Rate Articles: complete
    Status: ✅ Step 5 rate_articles completed successfully! Rated 174 articles with av

In [35]:
state.print_workflow_status()



📊 WORKFLOW STATUS
📈 Progress: 100.0% (9/9 complete)
🎉 All steps complete!

📋 Step Details:
  ✅ Step 0: Gather URLs            | complete
      📝 Status: ✅ Step 1 gather_urls completed successfully! Gathered 838 articles from 17 sources.

📊 Articles stored in persistent state: 702
  ✅ Step 1: Filter URLs            | complete
      📝 Status: ✅ Step 2 filter_urls completed successfully! Removed 347 duplicate URLs, classified 355 new articles, found 177 AI-related.
  ✅ Step 2: Download Articles      | complete
      📝 Status: ✅ Step 3 download_articles completed successfully! Total articles: 355; AI-related articles: 177; HTML files: 148; Text files: 148
  ✅ Step 3: Extract Summaries      | complete
      📝 Status: ✅ Step 4 extract_summaries completed successfully! Generated AI-powered summaries for 177/177 articles.
💾 Summaries stored in headline DataFrame.
  ✅ Step 4: Rate Articles          | complete
      📝 Status: ✅ Step 5 rate_articles completed successfully! Rated 174 articles wit